## Amazon SageMaker Feature Store: Introduction to Feature Store

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-featurestore|feature_store_introduction.ipynb)

---

This notebook demonstrates how to get started with Feature Store, create feature groups, and ingest data into them. These feature groups are stored in your Feature Store.

Feature groups are resources that contain metadata for all data stored in your Feature Store. A feature group is a logical grouping of features, defined in the feature store to describe records. A feature group’s definition is composed of a list of feature definitions, a record identifier name, and configurations for its online and offline store. 

### Overview
1. Set up
2. Creating a feature group
3. Ingest data into a feature group

### Prerequisites
This notebook uses both `boto3` and Python SDK libraries, and the `Python 3 (Data Science)` kernel. This notebook works with Studio, Jupyter, and JupyterLab. 

#### Library dependencies:
* `sagemaker>=2.100.0`
* `numpy`
* `pandas`

#### Role requirements:
**IMPORTANT**: You must attach the following policies to your execution role:
* `AmazonS3FullAccess`
* `AmazonSageMakerFeatureStoreAccess`

![policy](images/feature-store-policy.png)

### Set up

In [1]:
# SageMaker Python SDK version 2.100.0 is required
# boto3 version 1.24.20 is required
import sagemaker
import boto3
import sys

!pip install 'sagemaker>=2.100.0'
!pip install 'boto3>=1.24.20'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
import pandas as pd
import numpy as np
import io
from sagemaker.session import Session
from sagemaker import get_execution_role

prefix = "sagemaker-featurestore-introduction"
role = get_execution_role()

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
s3_bucket_name = sagemaker_session.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### Inspect your data
In this notebook example we ingest synthetic data. We read from `./data/feature_store_introduction_customer.csv` and `./data/feature_store_introduction_orders.csv`.

In [3]:
customer_data = pd.read_csv("data/feature_store_introduction_customer.csv")
orders_data = pd.read_csv("data/feature_store_introduction_orders.csv")

In [4]:
customer_data.head()

,customer_id,city_code,state_code,country_code
0,573291,1,49,2
1,109382,2,40,2
2,828400,3,31,2
3,124013,4,5,2


In [5]:
orders_data.head()

,customer_id,order_id,order_status,store_id
0,573291,4132,1,303
1,109382,5724,0,201
2,828400,1942,0,431
3,124013,6782,1,213


Below is an illustration on the steps the data goes through before it is ingested into a Feature Store. In this notebook, we illustrate the use-case where you have data from multiple sources and want to store them independently in a feature store. Our example considers data from a data warehouse (customer data), and data from a real-time streaming service (order data). 

![data flow](images/feature_store_data_ingest.svg)

### Create a feature group

We first start by creating feature group names for customer_data and orders_data. Following this, we create two Feature Groups, one for `customer_data` and another for `orders_data`

In [6]:
from time import gmtime, strftime, sleep

customers_feature_group_name = "customers-feature-group-" + strftime("%d-%H-%M-%S", gmtime())
orders_feature_group_name = "orders-feature-group-" + strftime("%d-%H-%M-%S", gmtime())

Instantiate a FeatureGroup object for customers_data and orders_data. 

In [7]:
from sagemaker.feature_store.feature_group import FeatureGroup

customers_feature_group = FeatureGroup(
    name=customers_feature_group_name, sagemaker_session=sagemaker_session
)
orders_feature_group = FeatureGroup(
    name=orders_feature_group_name, sagemaker_session=sagemaker_session
)

In [8]:
import time

current_time_sec = int(round(time.time()))

record_identifier_feature_name = "customer_id"

Append `EventTime` feature to your data frame. This parameter is required, and time stamps each data point.

In [9]:
customer_data["EventTime"] = pd.Series([current_time_sec] * len(customer_data), dtype="float64")
orders_data["EventTime"] = pd.Series([current_time_sec] * len(orders_data), dtype="float64")

Load feature definitions to your feature group. 

In [10]:
customers_feature_group.load_feature_definitions(data_frame=customer_data)
orders_feature_group.load_feature_definitions(data_frame=orders_data)

[FeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='order_id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='order_status', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='store_id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='EventTime', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None)]

Below we call create to create two feature groups, customers_feature_group and orders_feature_group respectively

In [11]:
customers_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

orders_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:624045005200:feature-group/orders-feature-group-22-08-57-48',
 'ResponseMetadata': {'RequestId': 'c0822289-fdd2-4ed6-bde2-e06faabe0aad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c0822289-fdd2-4ed6-bde2-e06faabe0aad',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '109',
   'date': 'Mon, 22 Jan 2024 08:57:55 GMT'},
  'RetryAttempts': 3}}

To confirm that your FeatureGroup has been created we use `DescribeFeatureGroup` and `ListFeatureGroups` APIs to display the created FeatureGroup.

In [12]:
customers_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:624045005200:feature-group/customers-feature-group-22-08-57-48',
 'FeatureGroupName': 'customers-feature-group-22-08-57-48',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'Integral'},
  {'FeatureName': 'city_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'state_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'country_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'EventTime', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2024, 1, 22, 8, 57, 53, 191000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-624045005200/sagemaker-featurestore-introduction',
   'ResolvedOutputS3Uri': 's3://sagemaker-us-east-1-624045005200/sagemaker-featurestore-introduction/624045005200/sagemaker/us-east-1/offline-sto

In [13]:
orders_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:624045005200:feature-group/orders-feature-group-22-08-57-48',
 'FeatureGroupName': 'orders-feature-group-22-08-57-48',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'Integral'},
  {'FeatureName': 'order_id', 'FeatureType': 'Integral'},
  {'FeatureName': 'order_status', 'FeatureType': 'Integral'},
  {'FeatureName': 'store_id', 'FeatureType': 'Integral'},
  {'FeatureName': 'EventTime', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2024, 1, 22, 8, 57, 55, 631000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-624045005200/sagemaker-featurestore-introduction',
   'ResolvedOutputS3Uri': 's3://sagemaker-us-east-1-624045005200/sagemaker-featurestore-introduction/624045005200/sagemaker/us-east-1/offline-store/orders

In [14]:
sagemaker_session.boto_session.client(
    "sagemaker", region_name=region
).list_feature_groups()  # We use the boto client to list FeatureGroups

{'FeatureGroupSummaries': [{'FeatureGroupName': 'orders-feature-group-22-08-57-48',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:624045005200:feature-group/orders-feature-group-22-08-57-48',
   'CreationTime': datetime.datetime(2024, 1, 22, 8, 57, 55, 631000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Creating'},
  {'FeatureGroupName': 'jjjj',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:624045005200:feature-group/jjjj',
   'CreationTime': datetime.datetime(2024, 1, 19, 5, 50, 14, 1000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'customers-feature-group-22-08-57-48',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:624045005200:feature-group/customers-feature-group-22-08-57-48',
   'CreationTime': datetime.datetime(2024, 1, 22, 8, 57, 53, 191000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Creating'}],
 'ResponseMetadata': {'RequestId': '63ec7148-1d3b-4666-8721-4d6b458d2067',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-request

In [15]:
def check_feature_group_status(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group to be Created")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group.name} successfully created.")


check_feature_group_status(customers_feature_group)
check_feature_group_status(orders_feature_group)

Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
FeatureGroup customers-feature-group-22-08-57-48 successfully created.
Waiting for Feature Group to be Created
FeatureGroup orders-feature-group-22-08-57-48 successfully created.


### Add metadata to a feature

We can add searchable metadata fields to FeatureGroup features by using the `UpdateFeatureMetadata` API. The currently supported metadata fields are `description` and `parameters`.

In [16]:
from sagemaker.feature_store.inputs import FeatureParameter

customers_feature_group.update_feature_metadata(
    feature_name="customer_id",
    description="The ID of a customer. It is also used in orders_feature_group.",
    parameter_additions=[FeatureParameter("idType", "primaryKey")],
)

{'ResponseMetadata': {'RequestId': 'd6113308-55b2-4743-91bc-a261037d807c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd6113308-55b2-4743-91bc-a261037d807c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 22 Jan 2024 08:58:15 GMT'},
  'RetryAttempts': 0}}

To view feature metadata, we can use `DescribeFeatureMetadata` to display that feature.

In [17]:
customers_feature_group.describe_feature_metadata(feature_name="customer_id")

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:624045005200:feature-group/customers-feature-group-22-08-57-48',
 'FeatureGroupName': 'customers-feature-group-22-08-57-48',
 'FeatureName': 'customer_id',
 'FeatureType': 'Integral',
 'CreationTime': datetime.datetime(2024, 1, 22, 8, 57, 53, 191000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 1, 22, 8, 58, 16, 690000, tzinfo=tzlocal()),
 'Description': 'The ID of a customer. It is also used in orders_feature_group.',
 'Parameters': [{'Key': 'idType', 'Value': 'primaryKey'}],
 'ResponseMetadata': {'RequestId': '5ca039cc-d183-4d13-ac8e-c28a6061d5c9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5ca039cc-d183-4d13-ac8e-c28a6061d5c9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '421',
   'date': 'Mon, 22 Jan 2024 08:58:16 GMT'},
  'RetryAttempts': 0}}

Feature metadata fields are searchable. We use `search` API to find features with metadata that matches some search criteria.

In [18]:
sagemaker_session.boto_session.client("sagemaker", region_name=region).search(
    Resource="FeatureMetadata",
    SearchExpression={
        "Filters": [
            {
                "Name": "FeatureGroupName",
                "Operator": "Contains",
                "Value": "customers-feature-group-",
            },
            {"Name": "Parameters.idType", "Operator": "Equals", "Value": "primaryKey"},
        ]
    },
)  # We use the boto client to search

{'Results': [],
 'ResponseMetadata': {'RequestId': '4e267efd-84a3-45ce-b3b0-6bdbdefca5d5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4e267efd-84a3-45ce-b3b0-6bdbdefca5d5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '14',
   'date': 'Mon, 22 Jan 2024 08:58:16 GMT'},
  'RetryAttempts': 0}}

### Ingest data into a feature group

We can put data into the FeatureGroup by using the `PutRecord` API. It will take < 1 minute to ingest data.

In [19]:
customers_feature_group.ingest(data_frame=customer_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='customers-feature-group-22-08-57-48', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f697dd4b1c0>, sagemaker_session=<sagemaker.session.Session object at 0x7f697ed58d60>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f697e7e8c70>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [20]:
orders_feature_group.ingest(data_frame=orders_data, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='orders-feature-group-22-08-57-48', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f697dd4b1c0>, sagemaker_session=<sagemaker.session.Session object at 0x7f697ed58d60>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f697e7e8250>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

Using an arbitrary customer record ID, 573291 we use `get_record` to check that the data has been ingested into the feature group.

In [21]:
customer_id = 573291
sample_record = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=customers_feature_group_name, RecordIdentifierValueAsString=str(customer_id)
)

In [22]:
sample_record

{'ResponseMetadata': {'RequestId': 'eb576181-5ef3-416d-8ee9-f8320ffc754c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eb576181-5ef3-416d-8ee9-f8320ffc754c',
   'content-type': 'application/json',
   'content-length': '417',
   'date': 'Mon, 22 Jan 2024 08:58:18 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'customer_id', 'ValueAsString': '573291'},
  {'FeatureName': 'city_code', 'ValueAsString': '1'},
  {'FeatureName': 'state_code', 'ValueAsString': '49'},
  {'FeatureName': 'country_code', 'ValueAsString': '2'},
  {'FeatureName': 'EventTime', 'ValueAsString': '1705913871.0'}]}

We use `batch_get_record` to check that all data has been ingested into two feature groups by providing customer IDs.

In [23]:
all_records = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": customers_feature_group_name,
            "RecordIdentifiersValueAsString": ["573291", "109382", "828400", "124013"],
        },
        {
            "FeatureGroupName": orders_feature_group_name,
            "RecordIdentifiersValueAsString": ["573291", "109382", "828400", "124013"],
        },
    ]
)

In [24]:
all_records

{'ResponseMetadata': {'RequestId': '58650ea4-8a21-4406-b2e2-26cc57d609f2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '58650ea4-8a21-4406-b2e2-26cc57d609f2',
   'content-type': 'application/json',
   'content-length': '4172',
   'date': 'Mon, 22 Jan 2024 08:58:20 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'customers-feature-group-22-08-57-48',
   'RecordIdentifierValueAsString': '573291',
   'Record': [{'FeatureName': 'customer_id', 'ValueAsString': '573291'},
    {'FeatureName': 'city_code', 'ValueAsString': '1'},
    {'FeatureName': 'state_code', 'ValueAsString': '49'},
    {'FeatureName': 'country_code', 'ValueAsString': '2'},
    {'FeatureName': 'EventTime', 'ValueAsString': '1705913871.0'}]},
  {'FeatureGroupName': 'customers-feature-group-22-08-57-48',
   'RecordIdentifierValueAsString': '109382',
   'Record': [{'FeatureName': 'customer_id', 'ValueAsString': '109382'},
    {'FeatureName': 'city_code', 'ValueAsString': '2'},
    {'FeatureNa

### Add features to a feature group

If we want to update a FeatureGroup that has done the data ingestion, we can use the `UpdateFeatureGroup` API and then re-ingest data by using the updated dataset.

In [25]:
from sagemaker.feature_store.feature_definition import StringFeatureDefinition

customers_feature_group.update(
    feature_additions=[StringFeatureDefinition("email"), StringFeatureDefinition("name")]
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:624045005200:feature-group/customers-feature-group-22-08-57-48',
 'ResponseMetadata': {'RequestId': 'aaecd46d-0413-41ec-9944-98aaa08f7d00',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aaecd46d-0413-41ec-9944-98aaa08f7d00',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '112',
   'date': 'Mon, 22 Jan 2024 08:58:21 GMT'},
  'RetryAttempts': 0}}

Verify the FeatureGroup has been updated successfully or not.

In [26]:
def check_last_update_status(feature_group):
    last_update_status = feature_group.describe().get("LastUpdateStatus")["Status"]
    while last_update_status == "InProgress":
        print("Waiting for FeatureGroup to be updated")
        time.sleep(5)
        last_update_status = feature_group.describe().get("LastUpdateStatus")
    if last_update_status == "Successful":
        print(f"FeatureGroup {feature_group.name} successfully updated.")
    else:
        print(
            f"FeatureGroup {feature_group.name} updated failed. The LastUpdateStatus is"
            + str(last_update_status)
        )


check_last_update_status(customers_feature_group)

Waiting for FeatureGroup to be updated
FeatureGroup customers-feature-group-22-08-57-48 updated failed. The LastUpdateStatus is{'Status': 'InProgress'}


Inspect the new dataset.

In [27]:
customer_data_updated = pd.read_csv("data/feature_store_introduction_customer_updated.csv")

In [28]:
customer_data_updated.head()

,customer_id,city_code,state_code,country_code,email,name
0,573291,1,49,2,john.lee@gmail.com,John Lee
1,109382,2,40,2,olivequil@gmail.com,Olive Quil
2,828400,3,31,2,liz.knee@gmail.com,Liz Knee
3,124013,4,5,2,eileenbook@gmail.com,Eileen Book


Append `EventTime` feature to your data frame again.

In [29]:
customer_data_updated["EventTime"] = pd.Series(
    [current_time_sec] * len(customer_data), dtype="float64"
)

Ingest the new dataset.

In [32]:
customers_feature_group.ingest(data_frame=customer_data_updated, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='customers-feature-group-22-08-57-48', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f697dd4b1c0>, sagemaker_session=<sagemaker.session.Session object at 0x7f697ed58d60>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f697e777700>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

Use `batch_get_record` again to check that all updated data has been ingested into `customers_feature_group` by providing customer IDs.

In [33]:
updated_customers_records = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": customers_feature_group_name,
            "RecordIdentifiersValueAsString": ["573291", "109382", "828400", "124013"],
        }
    ]
)

In [34]:
updated_customers_records

{'ResponseMetadata': {'RequestId': '155f42b8-dba4-4017-86cb-9468f3f20460',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '155f42b8-dba4-4017-86cb-9468f3f20460',
   'content-type': 'application/json',
   'content-length': '2768',
   'date': 'Mon, 22 Jan 2024 08:58:47 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'customers-feature-group-22-08-57-48',
   'RecordIdentifierValueAsString': '573291',
   'Record': [{'FeatureName': 'customer_id', 'ValueAsString': '573291'},
    {'FeatureName': 'city_code', 'ValueAsString': '1'},
    {'FeatureName': 'state_code', 'ValueAsString': '49'},
    {'FeatureName': 'country_code', 'ValueAsString': '2'},
    {'FeatureName': 'EventTime', 'ValueAsString': '1705913871.0'},
    {'FeatureName': 'email', 'ValueAsString': 'john.lee@gmail.com'},
    {'FeatureName': 'name', 'ValueAsString': 'John Lee'}]},
  {'FeatureGroupName': 'customers-feature-group-22-08-57-48',
   'RecordIdentifierValueAsString': '109382',
   'Record': [{'F

### Clean up
Here we remove the Feature Groups we created. 

In [35]:
customers_feature_group.delete()
orders_feature_group.delete()

### Next steps

In this notebook you learned how to quickly get started with Feature Store and now know how to create feature groups, and ingest data into them.

For an advanced example on how to use Feature Store for a Fraud Detection use-case, see [Fraud Detection with Feature Store](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-featurestore/sagemaker_featurestore_fraud_detection_python_sdk.html).

For detailed information about Feature Store, see the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html).


### Programmers note

In this notebook we used a variety of different API calls. Most of them are accessible through the Python SDK, however some only exist within `boto3`. You can invoke the Python SDK API calls directly on your Feature Store objects, whereas to invoke API calls that exist within `boto3`, you must first access a boto client through your boto and sagemaker sessions: e.g. `sagemaker_session.boto_session.client()`.

Below we list API calls used in this notebook that exist within the Python SDK and ones that exist in `boto3` for your reference. 

#### Python SDK API Calls
* `describe()`
* `ingest()`
* `delete()`
* `create()`
* `load_feature_definitions()`
* `update()`
* `update_feature_metadata()`
* `describe_feature_metadata()`

#### Boto3 API Calls
* `list_feature_groups()`
* `get_record()`
* `batch_get_record()`
* `search()`


## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-featurestore|feature_store_introduction.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-featurestore|feature_store_introduction.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-featurestore|feature_store_introduction.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-featurestore|feature_store_introduction.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-featurestore|feature_store_introduction.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-featurestore|feature_store_introduction.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-featurestore|feature_store_introduction.ipynb)
